In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
# fix dependence of id3
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator
from id3 import export_graphviz

In [2]:
# load raw data from csv file
raw = np.genfromtxt('hour.csv', delimiter = ',')
print(raw)

[[       nan        nan        nan ...        nan        nan        nan]
 [1.0000e+00        nan 1.0000e+00 ... 3.0000e+00 1.3000e+01 1.6000e+01]
 [2.0000e+00        nan 1.0000e+00 ... 8.0000e+00 3.2000e+01 4.0000e+01]
 ...
 [1.7377e+04        nan 1.0000e+00 ... 7.0000e+00 8.3000e+01 9.0000e+01]
 [1.7378e+04        nan 1.0000e+00 ... 1.3000e+01 4.8000e+01 6.1000e+01]
 [1.7379e+04        nan 1.0000e+00 ... 1.2000e+01 3.7000e+01 4.9000e+01]]


In [3]:
# read raw data into target/feature by discrete form
# target
cnt_median = np.median(raw[1:, -1:])
target = []
for i in raw[1:, -1:]:
    if i < cnt_median:
        target.append('Grow')
    else:
        target.append('Drop')
target = np.array(target)
print(target)

['Grow' 'Grow' 'Grow' ... 'Grow' 'Grow' 'Grow']


In [4]:
# features considered
feature_name = ['Season', 'Period of day', 'Holiday', 'Weather', 'Temperature', 'Humidity', 'Wind speed'] 
# cal median (better than mean)
temp_median = np.median(raw[1:, 10])
hum_median = np.median(raw[1:, 12])
wind_median = np.median(raw[1:, 13])
# Generate feature data 
feature = [[], [], [], [], [], [], []]
# A little mistake, use transpose below
for i in raw[1:]:
    # season
    if i[2] == 1:
        feature[0].append('Spring')
    elif i[2] == 2:
        feature[0].append('Summer')
    elif i[2] == 3:
        feature[0].append('Autumn')
    else:
        feature[0].append('Winter')
    # hour
    if i[5] < 6:
        feature[1].append('Early morning')
    elif i[5] < 12:
        feature[1].append('Morning')
    elif i[5] < 18:
        feature[1].append('Afternoon')
    else:
        feature[1].append('Evening')
    # holiday
    if i[6] == 1:
        feature[2].append('Yes')
    else:
        feature[2].append('No')
    '''
    # weekday
    cause the ugly tree
    if i[7] == 0:
        feature[3].append('Sun')
    elif i[7] == 1:
        feature[3].append('Mon')
    elif i[7] == 2:
        feature[3].append('Tue')
    elif i[7] == 3:
        feature[3].append('Wed') 
    elif i[7] == 4:
        feature[3].append('Thu')
    elif i[7] == 5:
        feature[3].append('Fri') 
    else:
        feature[3].append('Sat')
    '''
    # weather
    if i[9] == 1:
        feature[3].append('Best')
    elif i[9] == 2:
        feature[3].append('Good')
    elif i[9] == 3:
        feature[3].append('Bad')
    else:
        feature[3].append('Very bad')
    # temp
    if i[10] > temp_median:
        feature[4].append(f'>{int(temp_median*41)}')
    else:
        feature[4].append(f'<{int(temp_median*41)}')
    # humidity
    if i[12] > hum_median:
        feature[5].append(f'>{int(hum_median*100)}')
    else:
        feature[5].append(f'<{int(hum_median*100)}')
    # wind speed
    if i[13] > wind_median:
        feature[6].append(f'>{int(wind_median*67)}')
    else:
        feature[6].append(f'<{int(wind_median*67)}')
feature = np.array(feature).T
print(feature)

[['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ...
 ['Spring' 'Evening' 'No' ... '<20' '<63' '<12']
 ['Spring' 'Evening' 'No' ... '<20' '<63' '<12']
 ['Spring' 'Evening' 'No' ... '<20' '>63' '<12']]


In [5]:
# split train and test dataset
f_train, f_test, t_train, t_test = train_test_split(feature, target)
# build tree
model = Id3Estimator(max_depth=3)
# 6 is best but the tree is ugly
model.fit(f_train, t_train)

/Users/apple/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  return f(**kwargs)


Id3Estimator(max_depth=3)

In [6]:
# checking for adjusting
predict = model.predict(f_test)
print(predict)
print(t_test)
count = 0
for a, b in zip(predict, t_test):
    if a == b:
        count += 1
count/len(predict)
# ~0.8

['Drop' 'Drop' 'Drop' ... 'Grow' 'Drop' 'Drop']
['Drop' 'Drop' 'Drop' ... 'Grow' 'Grow' 'Drop']


/Users/apple/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  return f(**kwargs)


0.8138089758342922

In [7]:
# save
export_graphviz(model.tree_, 'tree.dot', feature_name, extensive=True)

<_io.TextIOWrapper name='tree.dot' mode='w' encoding='utf8'>

In [8]:
#     convert to pdf/png use pydot
#         $dot -Tpdf tree.dot -o tree.pdf
#         $dot -Tpng tree.dot -o tree.png